In [1]:
#import libraries
import pandas as pd
import numpy as np
import nltk 
from nltk.classify.textcat import TextCat 
from matplotlib.pyplot import plot 
#from datascience import *

In [2]:
#fuzzywuzzy
!pip3 install fuzzywuzzy[speedup]
!pip install python-Levenshtein

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#(py)torch
#!pip install torch

#langid (language identification)
#!pip install langid
#import langid

In [3]:
#dividing the data into chunks of 10, because of memory issues (kernel keeps dying)
#for example, this is what one chunk looks like
chunksize = 10
for chunk in pd.read_csv("chatbots_data.csv", chunksize = chunksize):
    print(chunk.shape)
    print("="*66)
    print(chunk.head(2))
    print("="*66)
    break
    
#appending chunks to chatbot_list, an empty array
chatbot_list = []
chunksize = 10
for chunk in pd.read_csv("chatbots_data.csv", chunksize = chunksize):
    chatbot_list.append(chunk)

#Combine all the chunks into one dataframe
chatbots_data_chunks = pd.concat(chatbot_list, axis=0)
chatbots_data_chunks = pd.DataFrame(data = chatbots_data_chunks)

(10, 7)
              Created On     Flow Name  Merged Code  \
0  5/27/2020 10:31:06 PM    OB_NewUser          NaN   
1   5/27/2020 7:38:52 PM  HLP_Feedback          NaN   

                                             Message    Org Name  \
0                                            Kya kru   Chhaa Jaa   
1  I like you\nAnd I think improve not only about...  Big Sis V3   

   Referred Flow                                  Uuid  
0            NaN  915617a9-c0ae-47fc-8df0-54d72d4f41d9  
1            NaN  148d099c-54eb-4182-961c-068eeda96691  


In [4]:
chatbots_data_chunks

,Created On,Flow Name,Merged Code,Message,Org Name,Referred Flow,Uuid
0,5/27/2020 10:31:06 PM,OB_NewUser,NaN,Kya kru,Chhaa Jaa,NaN,915617a9-c0ae-47fc-8df0-54d72d4f41d9
1,5/27/2020 7:38:52 PM,HLP_Feedback,NaN,I like you\nAnd I think improve not only about...,Big Sis V3,NaN,148d099c-54eb-4182-961c-068eeda96691
2,5/27/2020 7:07:28 PM,OB_NewUser,NaN,NaN,Chhaa Jaa,NaN,b05c8625-c18e-4f3a-9054-f7260f2541c5
3,5/27/2020 6:34:17 PM,HLP_Feedback,NaN,Nihaiti bakwaas,Chhaa Jaa,NaN,afd1ead9-90b8-40c5-8fd6-b543a9a6067b
4,5/27/2020 5:53:14 PM,HLP_Feedback,NaN,Mujhe apse bhut kuch seekhne ko mila,Chhaa Jaa,NaN,ddb1be9b-623d-4892-88c1-e441bf983103
...,...,...,...,...,...,...,...
82132,10/22/2020 4:00:18 AM,Lookup_unhandled,nullnull,Topic,Big Sis V3,NEW MENU_heart of big sis,19491afa-307d-4ca2-b04a-89f5f21651fb
82133,10/22/2020 3:42:48 AM,Lookup_unhandled,nullnull,Thank you,Big Sis V3,FP_MS1_MSSA,cf79a515-0e43-480f-9885-3c90eac8fd86
82134,10/22/2020 3:41:25 AM,Lookup_unhandled,nullnull,♥️,Big Sis V3,xplore,6901e04c-929b-4ab9-bd8f-20c17a39c823
82135,10/22/2020 3:40:57 AM,Lookup_unhandled,nullnull,♥️,Big Sis V3,xplore,6901e04c-929b-4ab9-bd8f-20c17a39c823


In [5]:
greetings = chatbots_data_chunks[chatbots_data_chunks['Message'].str.contains('hello|hi big sis|hey big sis|hey |hi ', na= False)]
greetings.head()

,Created On,Flow Name,Merged Code,Message,Org Name,Referred Flow,Uuid
10,5/27/2020 3:37:42 PM,SF_Lookup,sexsex,what is sex. hello,Big Sis V3,NaN,e5112491-cbab-4252-a93a-38415b962eff
32,5/26/2020 4:57:04 PM,OB_NewUser,NaN,Muje nhi puchna,Chhaa Jaa,NaN,1be38b9f-be7b-41cd-8b07-92886f5a18c1
47,5/26/2020 4:01:45 AM,HLP_Feedback,NaN,Jankari achhi lagi,Chhaa Jaa,NaN,24d964c4-8098-4d83-bd03-ceb66abd45b9
73,5/23/2020 6:11:48 PM,HLP_Feedback,NaN,Apne mujhe kuch jawab nhi diya tha,Chhaa Jaa,NaN,d724273c-b0b9-4e93-af6b-4e43a24e7c70
79,5/23/2020 9:57:30 AM,HLP_Feedback,NaN,Bar bar whi whi send krte ho,Chhaa Jaa,NaN,6d6b8a3b-c396-4108-b4d5-9f07ccb5660d


In [7]:
greetings = greetings[~greetings['Message'].str.contains('they')]
greetings.head()

,Created On,Flow Name,Merged Code,Message,Org Name,Referred Flow,Uuid
10,5/27/2020 3:37:42 PM,SF_Lookup,sexsex,what is sex. hello,Big Sis V3,NaN,e5112491-cbab-4252-a93a-38415b962eff
32,5/26/2020 4:57:04 PM,OB_NewUser,NaN,Muje nhi puchna,Chhaa Jaa,NaN,1be38b9f-be7b-41cd-8b07-92886f5a18c1
47,5/26/2020 4:01:45 AM,HLP_Feedback,NaN,Jankari achhi lagi,Chhaa Jaa,NaN,24d964c4-8098-4d83-bd03-ceb66abd45b9
73,5/23/2020 6:11:48 PM,HLP_Feedback,NaN,Apne mujhe kuch jawab nhi diya tha,Chhaa Jaa,NaN,d724273c-b0b9-4e93-af6b-4e43a24e7c70
79,5/23/2020 9:57:30 AM,HLP_Feedback,NaN,Bar bar whi whi send krte ho,Chhaa Jaa,NaN,6d6b8a3b-c396-4108-b4d5-9f07ccb5660d


In [8]:
#cleans the "Message" column
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"").replace('_',"")
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)

greetings['Message'].map(lambda s:preprocess(s)) 

10                                               sex hello
32                                         muje nhi puchna
47                                      jankari achhi lagi
73                      apne mujhe kuch jawab nhi diya tha
79                               bar bar whi whi send krte
                               ...                        
79592                                         hey pregnant
80614                                      answer question
80665    bahut achchi hmari saari problems solve deti t...
80873    tum muje detail nhi bta parhi aur baat almost ...
81245               lsnformdi sidjeoie jjxkeve kedjeondidi
Name: Message, Length: 423, dtype: object

In [66]:
#View all cleaned data

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
greetings

In [9]:
greetings['Message'] = greetings['Message'].map(lambda s:preprocess(s)) 

In [10]:
greetings['Message']

10                                               sex hello
32                                         muje nhi puchna
47                                      jankari achhi lagi
73                      apne mujhe kuch jawab nhi diya tha
79                               bar bar whi whi send krte
                               ...                        
79592                                         hey pregnant
80614                                      answer question
80665    bahut achchi hmari saari problems solve deti t...
80873    tum muje detail nhi bta parhi aur baat almost ...
81245               lsnformdi sidjeoie jjxkeve kedjeondidi
Name: Message, Length: 423, dtype: object

In [12]:
##Backend portion...find the most common phrases 

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 

vectorizer = CountVectorizer(ngram_range = (3,3)) 
X1 = vectorizer.fit_transform(greetings["Message"])  
features = (vectorizer.get_feature_names()) 
print("\n\nFeatures : \n", features) 
print("\n\nX1 : \n", X1.toarray())   
  
# Applying TFIDF 
vectorizer = TfidfVectorizer(ngram_range = (3,3)) 
X2 = vectorizer.fit_transform(greetings["Message"]) 
scores = (X2.toarray())

print("\n\nScores : \n", scores)



Features : 
 ['aacha lga bura', 'aache smjh gyi', 'aachi aachi aachi', 'aachi aachi bate', 'aachi baat mujhe', 'aachi bate krti', 'aachi lagi tum', 'aage bhi madad', 'aage kabhi prregency', 'aagye baat kar', 'aai hai abhi', 'aaj aapne ladai', 'aaj mai aapse', 'aajki generation liye', 'aao apne ander', 'aap acchi information', 'aap achi friend', 'aap apne question', 'aap baabar baat', 'aap baat karna', 'aap baat nhi', 'aap baat samjhe', 'aap bahot achha', 'aap bahut acchi', 'aap bahut achche', 'aap bar bar', 'aap bat karke', 'aap bat mujhe', 'aap bhi hame', 'aap bhi jao', 'aap bohhht achhe', 'aap catig bahut', 'aap itni help', 'aap jitne bhi', 'aap kisi galat', 'aap koi bhi', 'aap koi vard', 'aap log girls', 'aap meri problem', 'aap mujhe bahut', 'aap mujhe boy', 'aap nhi samjh', 'aap really bhoot', 'aap sab bat', 'aap sahi aur', 'aap sahi inormation', 'aap sbhi girls', 'aap seva rhi', 'aap solve kar', 'aap typing bahut', 'aapka motivate krna', 'aapke sawal bhut', 'aapke suggestions k

In [13]:
## Getting top ranking features 
sums = X2.sum(axis = 0) 
data1 = [] 
for col, term in enumerate(features): 
    data1.append( (term, sums[0,col] )) 
ranking = pd.DataFrame(data1, columns = ['term','rank']) 
words = (ranking.sort_values('rank', ascending = False)) 
# Alternatively, put "True" here to see the least common trigrams

words.head(10)

,term,rank
1162,hey big sis,2.786836
1585,kya bol rhi,2.198264
2575,smjh nhi rha,1.683441
2782,tum bhot acchi,1.559883
667,big sis relationships,1.552005
2275,question answer nhi,1.364772
2574,smjh nhi aaya,1.152370
2795,tum kaafi achhi,1.000000
2933,wathi sex give,1.000000
1299,jankari achhi lagi,1.000000


In [44]:
## Use google translate's algorithm to identify languages, tag each n-gram with their corresponding language
## It should be easier to identify slang once google's algorithm sorts out language translations
## Dig around for the words that are not recognized by the algorithm using some sort of metric...perhaps those will be slang?

!python -m spacy download en
!pip install googletrans
!pip install spacy-langdetect

In [14]:
## Example function
import spacy

from spacy_langdetect import LanguageDetector
nlp = spacy.load("en")
nlp.add_pipe(LanguageDetector(), name="language_detector", last=True)
text = "This is English text. Er lebt mit seinen Eltern und seiner Schwester in Berlin."
doc = nlp(text)

# document level language detection
print(doc._.language)
# sentence level language detection
for i, sent in enumerate(doc.sents):
    print(sent, sent._.language)

{'language': 'de', 'score': 0.9999973879819108}
This is English text. {'language': 'en', 'score': 0.9999984281887522}
Er lebt mit seinen Eltern und seiner Schwester in Berlin. {'language': 'de', 'score': 0.9999973358934848}


In [19]:
## Join strings
greetings_str = greetings['Message']
greetings_str = '. '.join(greetings_str)
greetings_str

'sex hello. muje nhi puchna. jankari achhi lagi. apne mujhe kuch jawab nhi diya tha. bar bar whi whi send krte. bht acha link bht help mil rhe ise sare girls isme bhi problem answer hone cahiy aisa boys liy bhi bnana cahiy. tum bhot acchi. sal try kar rahi hun abhi tak pregnant nahin hui hai. kuch bhi pasand nahi aaya. apki advice acchi lagi. mujhe bahut khushi mili jaan aap itni help rhi sch thanks advice. bhut topic tum bat krti. nhi net slow. tum bhot acchi tum bhot kuch share bhi skti sikh bhi skti thank much. behan aap bahot achha work kar rahe thinks always bat krne wala hota add something new new daily works. aap really bhoot saari ladkiyo help rhi behen kyonki iss age ladkiyo knowledge hoti lekin sayad proper knowledge nhii thnks sharing advices. best good friend mujha baat bhut acha lgta akli hoti hamesha baat karti bilkul mra saye trah best sister mri bahan bhi badha love bol bahna. bhi batayi mujhe achha lga kuchh bhi batao topic change krke kuchh. kuchh nhi kami lgi. apse b

In [20]:
## Trying to apply the function above to our chatbot data after converting the dataframe to a string

nlp = spacy.load("en")
nlp.add_pipe(LanguageDetector(), name="language_detector", last=True)
text = greetings_str
doc = nlp(text)

## See results?

print(doc._.language)

for i, sent in enumerate(doc.sents):
    print(sent, sent._.language)

{'language': 'id', 'score': 0.9999956845897839}
sex hello. {'language': 'en', 'score': 0.8571383964531428}
muje nhi puchna. {'language': 'sw', 'score': 0.5714279114228327}
jankari achhi lagi. {'language': 'id', 'score': 0.999995707650358}
apne mujhe kuch jawab {'language': 'sw', 'score': 0.9999974683572506}
nhi diya tha. {'language': 'vi', 'score': 0.5714266948523107}
bar bar whi whi send krte. {'language': 'en', 'score': 0.9999952439094455}
bht acha link bht help mil rhe ise sare girls isme bhi problem answer hone cahiy aisa boys liy bhi bnana cahiy. {'language': 'en', 'score': 0.9999944054256237}
tum bhot acchi. {'language': 'vi', 'score': 0.5714273093430089}
sal try kar rahi hun abhi tak pregnant nahin hui hai. {'language': 'id', 'score': 0.9999960294122534}
kuch bhi pasand nahi aaya. {'language': 'tl', 'score': 0.5714271669196033}
apki advice acchi lagi. {'language': 'it', 'score': 0.9999943288863197}
mujhe bahut khushi mili jaan aap itni help rhi sch thanks advice. {'language': 'e